In [2]:
import numpy as np
import glob
import random
from segment2d import *
import nibabel as nib
from ipywidgets import interact
import matplotlib.pyplot as plt
import albumentations as A
from torchvision import transforms
import os

import csv
import kornia as K
from tqdm import tqdm
from torch.utils.data import DataLoader
import yaml
import SimpleITK as sitk
import shutil

In [3]:
shutil.make_archive("nhatvinbig", 'zip', "/home/nhattm1/test_newcode", "nhatvinbig")

'/home/nhattm1/test_newcode/nhatvinbig.zip'

In [2]:
file_nolesion = glob.glob("data/msseg2016_test_center7_Patient_08_nolestion/Preprocessed_Data/*FLAIR*")

In [8]:
model = FCDenseNet(in_channels=cfg.DATA.INDIM_MODEL_MICCAI, n_classes=cfg.DATA.NUM_CLASS)

# List of subjects in test set
list_test_subject = sorted(glob.glob(f"data/MSSEG-Testing/Center*/*/Preprocessed_Data/*FLAIR*"))
file_nolesion = glob.glob("data/msseg2016_test_center7_Patient_08_nolestion/Preprocessed_Data/*FLAIR*")


# List of subjects in the training set
list_train_subject = sorted(glob.glob("data/msseg2016npz/*"))

# Create a MEGLoader object for the test data using the list_test_subject variable
test_dataset = MSSEG_Test_Loader(list_subject=list_test_subject)
# test_dataset = MSSEG_Test_Loader(file_nolesion)
segmenter = Segmenter(model, cfg.DATA.CLASS_WEIGHT, cfg.DATA.NUM_CLASS, 
                                        cfg.OPT.LEARNING_RATE, cfg.OPT.FACTOR_LR, cfg.OPT.PATIENCE_LR)
checkpoint = "/home/nhattm1/test_newcode/weights_msseg_tiramisu/ckpt_dice_0.7091.ckpt"
segmenter = Segmenter.load_from_checkpoint(checkpoint_path=checkpoint, model=model,
                                        class_weight=cfg.DATA.CLASS_WEIGHT,
                                            num_classes=cfg.DATA.NUM_CLASS, 
                                        learning_rate=cfg.OPT.LEARNING_RATE,
                                        factor_lr=cfg.OPT.FACTOR_LR, patience_lr=cfg.OPT.PATIENCE_LR)

PARAMS_TRAINER = {"accelerator":cfg.SYS.ACCELERATOR, "devices":cfg.SYS.DEVICES, 
                "benchmark":True, "enable_progress_bar":True, 
                    # "overfit_batches" :5,
                    "logger" : False,
                    # "callbacks" : [check_point_dice, lr_monitor, early_stopping],
                    "log_every_n_steps" :1, "num_sanity_val_steps":0, "max_epochs":cfg.TRAIN.EPOCHS,
                    "precision":cfg.SYS.MIX_PRECISION,
                    }

# Initialize a Trainer object with the specified parameters
trainer = pl.Trainer(**PARAMS_TRAINER)
trainer.test(segmenter, test_dataset, verbose=True)

Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

{'batch_score_val': 0.49926398172881514, 'batch_dice_val': 0.4818139120541373, 'batch_ppv_val': 0.673712064238829, 'batch_tpr_val': 0.37500004595587894, 'batch_lfpr_val': 0.7500015624902344, 'batch_ltpr_val': 0.6666722221296312, 'batch_vd_val': 0.4433823203395353}
{'batch_score_val': 0.8531114988183958, 'batch_dice_val': 0.8352916562040703, 'batch_ppv_val': 0.8239856609374986, 'batch_tpr_val': 0.8469122295513636, 'batch_lfpr_val': 0.20454726239258544, 'batch_ltpr_val': 0.9523810657593672, 'batch_vd_val': 0.02782398978615724}
{'batch_score_val': 0.8091058106050201, 'batch_dice_val': 0.8315583873976208, 'batch_ppv_val': 0.767194405613776, 'batch_tpr_val': 0.90771105155665, 'batch_lfpr_val': 0.23333461110898146, 'batch_ltpr_val': 0.8360658425150123, 'batch_vd_val': 0.183156504941478}
{'batch_score_val': 0.628077085568715, 'batch_dice_val': 0.8992559261029464, 'batch_ppv_val': 0.9040490920109422, 'batch_tpr_val': 0.8945133178720074, 'batch_lfpr_val': 0.6666674074057612, 'batch_ltpr_val': 0

[{}]

In [ ]:
# flair = nib.load("data/data_isbi_2015/training/training01/preprocessed/training01_01_flair_pp.nii").get_fdata()
# aug_flair = augmentation(image=image_flair)["image"]
def show_segmentation(index):
    try:
        plt.figure(figsize=(20, 20))
        plt.subplot(1, 5, 1),
        plt.imshow(padded_flair[...,index]),
        plt.subplot(1, 5, 2),
        plt.imshow(flair[...,index]),
        # plt.subplot(1, 5, 3),
        # plt.imshow(padded_t2[...,index]),
        # plt.subplot(1, 5, 4),
        # plt.imshow(padded_mask[...,index]),
        plt.show(),
    except:
        pass
    # plt.subplot(1, 5, 5),
    # plt.imshow(augmented_mask),
interact(show_segmentation, index=(0,500))

In [2]:
list_file_flair = sorted(glob.glob(f"data/msseg-2008-testing-nii/*/*FLAIR*"))


In [15]:
img1 = sitk.ReadImage("/home/nhattm1/test_newcode/data/msseg-2008-testing-nii/CHB_test1_Case01/CHB_test1_Case01_FLAIR_brain_bias_correction.nii.gz")
#get numpy array from SimpleITK image
img = sitk.GetArrayFromImage(img1).astype(np.uint8)
new_img = sitk.GetImageFromArray(img)

In [17]:
new_img.CopyInformation(img1)

In [16]:

# Use sitk.WriteImage to write the new image
output_filename = "abc.nhdr"  # Replace with your desired output file path and format
sitk.WriteImage(new_img, output_filename)

In [6]:
path_flair = list_file_flair[0]
id_subject = path_flair.split("/")[-2]
id_subject

'CHB_test1_Case01'

In [5]:
def show_segmentation(index):
    try:
        plt.figure(figsize=(20, 20))
        plt.subplot(1, 5, 1),
        plt.imshow(flair[...,index]),
        plt.subplot(1, 5, 2),
        plt.imshow(mask),
        # plt.subplot(1, 5, 3),
        # plt.imshow(padded_t2[...,index]),
        # plt.subplot(1, 5, 4),
        # plt.imshow(padded_mask[...,index]),
        plt.show(),
    except:
        pass
    # plt.subplot(1, 5, 5),
    # plt.imshow(augmented_mask),
interact(show_segmentation, index=(0,2))

interactive(children=(IntSlider(value=1, description='index', max=2), Output()), _dom_classes=('widget-interac…

<function __main__.show_segmentation(index)>